In [5]:
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path

In [6]:
MEMBERS = "./members.csv"
SONG_FEAT = "./song_extra_info.csv"
SONGS = "./songs.csv"
TRAIN = "./train.csv"

In [7]:
members_df = pd.read_csv(MEMBERS)
song_extra_df = pd.read_csv(SONG_FEAT)
songs_df = pd.read_csv(SONGS)
train_df = pd.read_csv(TRAIN)

# Getting prep members

In [8]:
members_df['registration_year'] = members_df['registration_init_time'].apply(lambda x: int(str(x)[:4]))
members_df['registration_month'] = members_df['registration_init_time'].apply(lambda x: int(str(x)[4:6]))
members_df['registration_day'] = members_df['registration_init_time'].apply(lambda x: int(str(x)[6:]))

members_df['expiration_year'] = members_df['expiration_date'].apply(lambda x: int(str(x)[:4]))
members_df['expiration_month'] = members_df['expiration_date'].apply(lambda x: int(str(x)[4:6]))
members_df['expiration_day'] = members_df['expiration_date'].apply(lambda x: int(str(x)[6:]))

members_df['bd'] = members_df['bd'].apply(lambda x: -x if x < 0 else x)
members_df['bd'] = members_df['bd'].apply(lambda x: x % 100)

members_df['nogender_noage'] = (members_df['bd'] == 0) & (members_df['gender'].isna()) # Люди, которые не указали ни пол ни возраст

#Заполним пропуски (те что равны нулю)
members_df['bd'] = members_df['bd'].replace(0,  members_df[members_df['bd'] != 0]['bd'].median())
members_df['gender'] = members_df['gender'].fillna(3) # Пусть возраст будет категориальным признаком, в котором тройка будет означать то что не указли пол

members_df['registration_init_time'] = pd.to_datetime(
    members_df['registration_init_time'], format='%Y%m%d'
)
members_df['expiration_date'] = pd.to_datetime(
    members_df['expiration_date'], format='%Y%m%d'
)
members_df['membership_duration'] = (
    members_df['expiration_date'] - members_df['registration_init_time']
).dt.days

In [12]:
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date,registration_year,registration_month,registration_day,expiration_year,expiration_month,expiration_day,nogender_noage,membership_duration
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,27,3,7,2011-08-20,2017-09-20,2011,8,20,2017,9,20,True,2223
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,27,3,7,2015-06-28,2017-06-22,2015,6,28,2017,6,22,True,725
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,27,3,4,2016-04-11,2017-07-12,2016,4,11,2017,7,12,True,457
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,27,3,9,2015-09-06,2015-09-07,2015,9,6,2015,9,7,True,1
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,27,3,4,2017-01-26,2017-06-13,2017,1,26,2017,6,13,True,138


In [ ]:
# members_df.to_csv("prep_members.csv")

# Getting prep train_df

In [15]:
train = train_df.copy()
train['source_type'] = train['source_type'].fillna("isnan")
train['source_screen_name'] = train['source_screen_name'].fillna("isnan")
train['source_system_tab'] = train['source_system_tab'].fillna("isnan")

top_songs = train.groupby(['song_id'])['target'].value_counts().reset_index( 
  name='Count').sort_values(['Count'], ascending=False)
top = top_songs.join(song_extra_df.set_index('song_id'),on='song_id',how='left')

no_tw_top = top[~top['isrc'].str[0:2].isin( ['TW','HK','CN'])]
train_no_chinese = train[train['song_id'].isin(no_tw_top['song_id'])]

In [23]:
train_no_chinese.shape

(3314966, 6)

In [22]:
train_no_chinese.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [24]:
# train_no_chinese.to_csv("prep_train_df.csv")

# Getting prep songs x songs_extra_info

In [25]:
merge_songs = songs_df.merge(song_extra_df, how = 'left', on = 'song_id')
merge_songs['genre_ids'] = merge_songs['genre_ids'].fillna(0)
merge_songs['composer'] = merge_songs['composer'].fillna('Unknown composer')
merge_songs['lyricist'] = merge_songs['lyricist'].fillna('Unknown lyricist')
merge_songs['language'] = merge_songs['language'].fillna(-1)
merge_songs['name'] = merge_songs['name'].fillna('Unknown name')
merge_songs['isrc'] = merge_songs['isrc'].fillna('Unknown isrc')
merge_songs['country'] = merge_songs['isrc'].apply(lambda x: x[:2] if "Unknown" not in x else "Unknown country")
merge_songs['organization'] = merge_songs['isrc'].apply(lambda x: x[2:5] if "Unknown" not in x else "Unknown org")
merge_songs['year_of_registration'] = merge_songs['isrc'].apply(lambda x: x[5:7] if "Unknown" not in x else "Unknown year")

In [29]:
merge_songs.shape

(2296320, 12)

In [28]:
merge_songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,name,isrc,country,organization,year_of_registration
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0,焚情,TWB531410010,TW,B53,14
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0,PLAYING WITH FIRE,Unknown isrc,Unknown country,Unknown org,Unknown year
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,Unknown composer,Unknown lyricist,31.0,SORRY| SORRY,Unknown isrc,Unknown country,Unknown org,Unknown year
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0,愛我的資格,TWC950206108,TW,C95,02
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0,Mary Had a Little Lamb,Unknown isrc,Unknown country,Unknown org,Unknown year


In [ ]:
# merge_songs.to_csv("prep_songs.csv")

# Train_df X Members by Andrew White

In [35]:
t = train_no_chinese.copy()
usr = members_df.copy()
almost_train_df = pd.merge(t, usr, on='msno', how='left')

In [36]:
almost_train_df.shape

(3314966, 20)

In [37]:
almost_train_df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,city,bd,gender,registered_via,registration_init_time,expiration_date,registration_year,registration_month,registration_day,expiration_year,expiration_month,expiration_day,nogender_noage,membership_duration
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,1,27,3,7,2012-01-02,2017-10-05,2012,1,2,2017,10,5,True,2103
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,13,24,female,9,2011-05-25,2017-09-11,2011,5,25,2017,9,11,False,2301
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,13,24,female,9,2011-05-25,2017-09-11,2011,5,25,2017,9,11,False,2301
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,13,24,female,9,2011-05-25,2017-09-11,2011,5,25,2017,9,11,False,2301
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,1,27,3,7,2012-01-02,2017-10-05,2012,1,2,2017,10,5,True,2103


In [42]:
from sklearn.preprocessing import LabelEncoder
train_data = almost_train_df.copy()

cols = ['msno', 'song_id', 'source_system_tab',	'source_screen_name', 'source_type']
encoders = {}

for col in cols:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    encoders[col] = le  # сохраняем энкодер

In [46]:
train_data.shape

(3314966, 20)

In [47]:
train_data.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,city,bd,gender,registered_via,registration_init_time,expiration_date,registration_year,registration_month,registration_day,expiration_year,expiration_month,expiration_day,nogender_noage,membership_duration
0,7640,60186,1,7,7,1,1,27,3,7,2012-01-02,2017-10-05,2012,1,2,2017,10,5,True,2103
1,16139,179889,4,8,5,1,13,24,female,9,2011-05-25,2017-09-11,2011,5,25,2017,9,11,False,2301
2,16139,97284,4,8,5,1,13,24,female,9,2011-05-25,2017-09-11,2011,5,25,2017,9,11,False,2301
3,16139,19141,4,8,5,1,13,24,female,9,2011-05-25,2017-09-11,2011,5,25,2017,9,11,False,2301
4,7640,26909,1,7,7,1,1,27,3,7,2012-01-02,2017-10-05,2012,1,2,2017,10,5,True,2103


In [48]:
train_data.to_csv("prep_train_data.csv")